In [1]:
import tensorflow as tf

In [ ]:
graph = tf.Graph()
with graph.as_default():
    mat1 = tf.constant([[3., 3.]])
    mat2 = tf.constant([[2.],[2.]])
    prod = tf.matmul(mat1, mat2)
 
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    result = session.run([prod])
    print result   

In [ ]:
#interactive Session
sess = tf.InteractiveSession()

x = tf.Variable([1.0, 2.0])
a = tf.constant([3.0, 3.0])

# Initialize 'x' using the run() method of its initializer op.
x.initializer.run()

#Add an op to subtract 'a' from 'x'.  Run it and print the result
sub = tf.sub(x, a)
print sub.eval()
# ==> [-2. -1.]

# Close the Session when we're done.
sess.close()


In [2]:
from six.moves import cPickle as pickle
import tensorflow as tf
import numpy as np

pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28), (200000,))
('Validation set', (10000, 28, 28), (10000,))
('Test set', (10000, 28, 28), (10000,))
('Training set', (200000, 784), (200000, 10))
('Validation set', (10000, 784), (10000, 10))
('Test set', (10000, 784), (10000, 10))


In [ ]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  hidden_units = 1024
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  weights2 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))  

  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  #an_relu = tf.nn.relu_layer(logits,weights2, biases2)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [13]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [19]:
batch_size = 128
image_size = 28
num_labels = 10

#interactive Session
sess = tf.InteractiveSession()

#tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
#tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
tf_valid_dataset = tf.constant(valid_dataset)
tf_test_dataset = tf.constant(test_dataset)

tf_train_data = tf.constant(train_dataset[0:batch_size,])
tf_train_label = tf.constant(train_labels[0:batch_size])


# Variables.
W_fc1 = weight_variable([image_size*image_size,1024])
b_fc1 = bias_variable([1024])

weights = weight_variable([1024, 10])
biases = bias_variable([10])

sess.run(tf.initialize_all_variables())

#img_flat = tf.reshape(tf_train_data,[-1,image_size*image_size*batch_size])



h_fc1 = tf.nn.relu(tf.matmul(tf_train_data, W_fc1) + b_fc1)


logits = tf.matmul(h_fc1, weights) + biases

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_label))
  
# Optimizer.
optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
# Predictions for the training, validation, and test data.
train_prediction = tf.nn.softmax(logits)


#logits = tf.matmul(tf_train_data, weights) + biases
# Close the Session when we're done.
#sess.close()
print(tf.shape(W_fc1))
print(tf.Tensor.get_shape(img_flat))
print(tf.Tensor.get_shape(h_fc1))
print(tf.Tensor.get_shape(logits))
print(tf.Tensor.get_shape(tf_train_label))
print(tf.Tensor.get_shape(train_prediction))
print(train_prediction.eval())
print('Test accuracy: %.1f%%' % accuracy(train_prediction.eval(), test_labels))
sess.close()

Tensor("Shape_9:0", shape=(2,), dtype=int32)
(1, 100352)
(128, 1024)
(128, 10)
(128, 10)
(128, 10)
[[  2.92259842e-01   3.72505039e-02   6.63592387e-03 ...,   1.52622029e-01
    1.22287398e-04   1.84597373e-01]
 [  5.54208338e-01   2.27210715e-01   1.11901539e-03 ...,   9.59831104e-03
    4.06179373e-04   7.66652003e-02]
 [  9.69206333e-01   1.46311498e-03   3.81272956e-04 ...,   8.21926165e-04
    4.55634494e-04   1.22775668e-02]
 ..., 
 [  1.42914504e-01   2.43221819e-01   9.15296434e-04 ...,   9.85275023e-03
    5.52959647e-03   2.49918457e-03]
 [  1.77854672e-01   2.49019964e-03   5.16269775e-03 ...,   3.62756252e-02
    7.60324579e-03   8.05609003e-02]
 [  1.23051628e-01   8.85636583e-02   1.81917790e-02 ...,   1.57107657e-03
    5.07578962e-02   7.58451596e-02]]
Test accuracy: 0.0%


/home/nyx/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:11: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


In [16]:
sess.close()